# Mortal Fibonacci Rabbits

## Problem

Recall the definition of the Fibonacci numbers from “Rabbits and Recurrence Relations”, which followed the recurrence relation $F_n=F_{n-1} +F_{n-2}$ and assumed that each pair of rabbits reaches maturity in one month and produces a single pair of offspring (one male, one female) each subsequent month.

Our aim is to somehow modify this recurrence relation to achieve a dynamic programming solution in the case that all rabbits die out after a fixed number of months.

_Given_: Positive integers $n≤100$ and $m≤20$.

Return: The total number of pairs of rabbits that will remain after the $n$-th month if all rabbits live for $m$ months.

**Sample Dataset**

    6 3

**Sample Output**

    4

## Solution

In this notebook we present one possible solution to this variation on the classic Fibonacci problem based on keeping track of the ages of the rabbit population. We will then optimize this solution with some tricks and the double ended queue data structure.

We start by carefully examining the problem statement. Just like in the regular Fibonacci rabbit problem, rabbits can reproduce only after their first month. And the new twist is that rabbits die after $m$ months. Picture a population of rabbits at a particular time $t$. If you looked at the ages of the rabbits you will find that they vary from 1 to $m$. Now if you look at the next generation $t+1$ you will find that every rabbit that was at least 2 months old has produced offspring, that every rabbit under $m$ months old has aged 1 month, and that however many rabbits were $m$ months old has died. Let's write that down in a list, assuming that rabbits die at age 4, and that we have $a, b, c$ and $d$ rabbits aged $1, 2, 3$, and $4$ months respectively at some time $t$.

    Population at time t: [a, b, c, d]
    
Again, at time $t+1$, $b, c$, and $d$ rabbits will have babies, $a, b$, and $c$ rabbits will age (jump a spot on the list), and $d$ rabbits will die:

    Population at time t+1 [b+c+d, a, b, c]
    
Let's code our first solution using a list. We start by initializing the a list of size $m$ with one rabbit in the first position and the rest zeros. Then for each generation we insert in the zero position the sum of every term in the list except the first one, and we remove the last entry.

In [1]:
def mortal_fibonacci_brute(n, m=1):
    # Prepare formats for printing output
    FMT = ('{:^10}{:^5d} | {:^9d} | {:^9d} | {:^9d} || {:^9d}')
    print('{:^15} | {:^9} | {:^9} | {:^9} || {:^9}'.format('Generation #', '1 mo.', '2 mo.', '3 mo.', 'tot'))
    print('-'*62)
    
    # Initialize list
    ages = [1] + [0]*(m-1)
    print(FMT.format('', 1, *ages, sum(ages)))
    
    # Compute the number of rabbits for each of the n generations
    for i in range(n-1):
        ages = [sum(ages[1:])] + ages[:-1]
        print(FMT.format('', i+2, *ages, sum(ages)))

    return sum(ages)

mortal_fibonacci_brute(6,3)

 Generation #   |   1 mo.   |   2 mo.   |   3 mo.   ||    tot   
--------------------------------------------------------------
            1   |     1     |     0     |     0     ||     1    
            2   |     0     |     1     |     0     ||     1    
            3   |     1     |     0     |     1     ||     2    
            4   |     1     |     1     |     0     ||     2    
            5   |     1     |     1     |     1     ||     3    
            6   |     2     |     1     |     1     ||     4    


4

We see that with this elegant approach we can obtain the correct solution. But how fast is our approach? For each generation we need to sum $m-1$ elements, and we have to do this for $n$ generations, so we expect our algorithm to have a $O(n*m)$ running time which is a little much. How can we bring it down?

It may not seem obvious with the small numbers used in the example above, but for bigger $m$ we are doing a lot of redundant summations. For example, if $m=10$, from $t$ to $t+1$, 9 out of the 10 elements in our list do not change! So we only need to keep track of how many rabbits died and how many were born in the last generation, and keep updating the sum with a little book keeping.

In [2]:
def mortal_fibonacci_nosum(n, m=1):
    # Initialize list
    ages = [1] + [0]*(m-1)
    
    # Compute the number of rabbits for each of the n generations
    tot = 0
    for i in range(n-1):
        new = ages[0]
        deaths = ages[-1]
        ages = [tot] + ages[:-1]
        tot = tot + new - deaths
        
    return sum(ages)

mortal_fibonacci_nosum(6,3)

4

Since accesing lists is _O_(1), we have gone from _O(n*m)_ down to _O(n)_... or have we? Actually there is still sometthing holding us back. When we update our age list we have to shift every element but the last one one position to the right, since we are essentially inserting something at the zero position. There is nothing much we can do about if we want to stick to lists, but there is a data structure that offers constant time insertions and deletions both from the end **and** its beginning: the double ended queue or *deque*. This extremely useful structure, which acts as a double linked list has slow lookup, insertion and deletion (from anywhere other than the head or tail) times but offers constant insertion and deletion and lookup times for the front and the end of the queue. We will take advantage of this data structure to reduce our running time.

There is another property of deques that will come in handy: specifying a maximum length for our queue. I a deque of fixed length is full and we try to insert something at the head, then the tail will be lost. So the statement ```ages = [tot] + ages[:-1]``` (which takes *O(m)* can be substitued by a simple ```ages.appendleft(tot)```, where ages is a deque, in constant time!

In [3]:
from collections import deque

def mortal_fibonacci_deque(n, m=1):
    # Initialize dequeue
    ages = deque([1] + [0]*(m-1), maxlen=m)
    
    # Compute the number of rabbits for each of the n generations
    tot = 0
    for i in range(n-1):
        new = ages[0]
        deaths = ages[-1]
        ages.appendleft(tot)
        tot = tot + new - deaths
        
    return sum(ages)

mortal_fibonacci_deque(6,3)

4

The difference in performance is outstanding. Let us time the functions using python's performance counter. First, let's clean up the first implementation of all the print statements used for explanation purposes.

In [4]:
def mortal_fibonacci_brute(n, m=1):
    # Initialize list
    ages = [1] + [0]*(m-1)
    
    # Compute the number of rabbits for each of the n generations
    for i in range(n-1):
        ages = [sum(ages[1:])] + ages[:-1]
    return sum(ages)

We are now ready to time the functions. We have seen that the first two solutions have a time complexity of _O(m*n)_ but the second one will have smaller constants since we are getting rid of the summations which was clearly the biggest bottleneck. The last solution runs in _O(n)_.

In [5]:
from time import perf_counter as pc

def time_fibonaccis(f1, f2, f3, *kwargs):
    print('Time taken to compute mortal_fibonacci_brute(%d, %d): ' % kwargs)
    t0 = pc()
    f1(*kwargs)
    print(pc() - t0)

    print('\nTime taken to compute mortal_fibonacci_nosum(%d, %d): ' % kwargs)
    t0 = pc()
    f2(*kwargs)
    print(pc() - t0)
    
    print('\nTime taken to compute mortal_fibonacci_deque(%d, %d): ' % kwargs)
    t0 = pc()
    f3(*kwargs)
    print(pc() - t0)
    
time_fibonaccis(mortal_fibonacci_brute, mortal_fibonacci_nosum, mortal_fibonacci_deque, 10000, 5000)

Time taken to compute mortal_fibonacci_brute(10000, 5000): 
11.407678648000001

Time taken to compute mortal_fibonacci_nosum(10000, 5000): 
0.6372917010012316

Time taken to compute mortal_fibonacci_deque(10000, 5000): 
0.009849602996837348
